In [ ]:
# In this session, we carry out a time-series forecasting to predict
# the monthly sales in January 2016, by first analyzing the monthly
# sales in the years 2013, 2014 and 2015.

# Check the project workspace.
import os
print("Project Workspace:", os.getcwd())
print()

# Import all necessary packages for the project.
import pandas as pan
import numpy as np
import statistics as myStats
import glob as glob
from matplotlib import pyplot as plt
plt.style.use("fivethirtyeight")
import statsmodels.api as myStatsModel
from pylab import rcParams
rcParams['figure.figsize'] = 18, 8
rcParams['text.color'] = 'k'
import itertools
import warnings
warnings.filterwarnings("ignore")
import matplotlib
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss

# Revome scientific notation to expand numbers in the dataset up to the
# second decimal place.
pan.set_option('display.float_format', lambda x: '%.4f' % x)

In [ ]:
# From the "Sale Date" data, we get the day, month and date columns for each transaction.
def fetchMonthAndYearColumns(dataFile):
    dataFile["Day"] = dataFile["Sale_Date"].apply(lambda month: month.split("-")[2])
    dataFile["Month"] = dataFile["Sale_Date"].apply(lambda month: month.split("-")[1])
    dataFile["Year"] = dataFile["Sale_Date"].apply(lambda year: year.split("-")[0])
    dataFile.drop(columns = ["Sale_Date"])
    return dataFile

# From the "Sale Date" data, we get the day, month and date columns for each transaction.
def aggregateDates(dataFile):
    dataFile["Sale_Date"] = pan.to_datetime(dataFile["Sale_Date"])
    dataFile = dataFile.groupby(by = ["Sale_Date"], as_index = False).agg({"Item_Value": "sum"}).rename(columns = {"Item_Value": "Total_Sales"})
    return dataFile

# Import data for each day and collate the data. For this, we create a function
# "importAndCollate()".
def importAndCollate():
    dataFileList = []
    yearList = ['2013', '2014', '2015']
    for year in yearList:
        fileNames = [fileName for fileName in glob.glob(year + "/*.csv", recursive = True)]
        for fileName in fileNames:
            dataFileList.append(pan.read_csv(fileName))
    yearDataFile = pan.concat(dataFileList, ignore_index = False)
    print("Size of Dataset:", yearDataFile.shape)
    print()
    return yearDataFile

In [ ]:
# Import all data for a year and collate it in one dataframe.
yearDataFile = importAndCollate()

In [ ]:
# Let's aggregate the data over each day to extract the total sales
# amount for each day.
aggregatedData = aggregateDates(yearDataFile)
aggregatedData["Sale_Date"] = pan.to_datetime(aggregatedData["Sale_Date"])
aggregatedData = aggregatedData.set_index('Sale_Date')
aggregatedData = aggregatedData.resample("MS").mean()

In [ ]:
# Next, we visualize the time series data.
aggregatedData.plot(figsize = (15, 6))
plt.show()

In [ ]:
# From the two plots, it is evident that there may be a pattern for
# the total sales generated over each month. Also, January 2015 seems to have
# incomplete data, i.e. it doesn't have records for all days of the month. Just
# 17 days. We will need to impute this using time-series analytics.
aggregatedData["No_of_Days"] = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31] * 3
aggregatedData["Total_Sales"] = aggregatedData["Total_Sales"] * aggregatedData["No_of_Days"]
display(aggregatedData)
aggregatedData = aggregatedData.drop(columns = ["No_of_Days"], axis = 1)
figure = aggregatedData.plot(figsize = (15, 6))
plt.xlabel("Months")
plt.ylabel("Total Sales")
plt.show()

In [ ]:
# We need to check the trend, seasonality, and noise in the data.
decomposition = myStatsModel.tsa.seasonal_decompose(aggregatedData, model = "additive")
fig = decomposition.plot()
plt.show()

In [ ]:
# Dickey-Fuller Test for Stationarity
dftest = adfuller(aggregatedData["Total_Sales"], autolag = "AIC")
dfoutput = pan.Series(dftest[0:4], index = ["Test Statistic", "p-value", "#Lags Used", "Number of Observations Used"])
for key, value in dftest[4].items():
    dfoutput["Critical Value (%s)"%key] = value
print(dfoutput)
print()

# KPSS Test for Stationarity
kpsstest = kpss(aggregatedData["Total_Sales"], regression='c')
kpss_output = pan.Series(kpsstest[0:3], index = ["Test Statistic", "p-value", "Lags Used"])
for key,value in kpsstest[3].items():
    kpss_output["Critical Value (%s)"%key] = value
print (kpss_output)
print()

In [ ]:
# We can see that the sales follow an upward trend, and are seasonal.
# Knowing this, we carry out time series forecasting using ARIMA, 
# (Auto-Regressive Integrated Moving Average).
# ARIMA models are denoted with the notation ARIMA(p, d, q). These three
# parameters account for seasonality, trend, and noise in data.
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = myStatsModel.tsa.statespace.SARIMAX(aggregatedData,
                                            order = param,
                                            seasonal_order = param_seasonal,
                                            enforce_stationarity = False,
                                            enforce_invertibility = False)
            results = mod.fit(disp = True)
            print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
            continue

In [ ]:
# We see that ARIMA(1, 1, 0)x(1, 1, 0, 12)12 - AIC:197.1350848946603 is the
# lowest value, and so we use these parameters to fit the ARIMA model for
# forecasting.
modelARIMA = myStatsModel.tsa.statespace.SARIMAX(aggregatedData,
                                order = (1, 1, 1),
                                seasonal_order = (1, 1, 0, 12),
                                enforce_stationarity = True,
                                enforce_invertibility = False)

results = modelARIMA.fit()
print(results.summary())
results.plot_diagnostics(figsize = (16, 8))
plt.show()

In [ ]:
# Now that we've fit our model, let's use it to make predictions.
pred = results.get_prediction(start = pan.to_datetime("2015-01-01"), end = pan.to_datetime("2015-12-01"), dynamic = True)
pred_ci = pred.conf_int()
ax = aggregatedData['2013':].plot(label = "Observed")
pred.predicted_mean.plot(ax = ax, label = 'One-step ahead Forecast', alpha = .7, figsize = (14, 7))
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color = 'k', alpha = .2)
ax.set_xlabel('Date')
ax.set_ylabel('Total Monthly Sales (in Dollars)')
plt.legend()
plt.show()

In [ ]:
# Use the model to forecast total sales for up to 12 steps ahead in time.
pred_uc = results.get_forecast(steps = 12)
pred_ci = pred_uc.conf_int()
ax = aggregatedData.plot(label = "Observed", figsize = (14, 7))
pred_uc.predicted_mean.plot(ax = ax, label = "Forecast")
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color = "k", alpha = 0.25)
ax.set_xlabel("Date")
ax.set_ylabel("Total Monthly Sales (in Dollars)")
plt.legend()
plt.show()

In [ ]:
print("Predict Sales in Dollars for 2016:")
display(pred_uc.predicted_mean)
print()
print("Error Margins for Predict Sales in Dollars for 2016:")
display(pred_uc.conf_int())

In [ ]:
predicted = pred.predicted_mean
truth = aggregatedData["Total_Sales"]["2015-01-01":]
mse = ((predicted - truth) ** 2).mean()
print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 2)))
print()
print('The Root Mean Squared Error of our forecasts is {}'.format(round(np.sqrt(mse), 2)))